In [ ]:
#how data are represented at each level (forward, backward, forward with padding on top) needs a little
    #experimentation to determine the best representation
    #also, is encoding at each layer really the best way? or just feeding the raw through?
    
#Outside web ips are going to be a problem/messy/noisy. Start by categorizing all outside ips by <OUTSIDE_IP>
    #instead of the ip address, or another 4 digit symbol to insert into the hex string.
    
#to help the models generalize more, for a given source ip address with probability p (say p = 0.1) 
    #use the token <OTHER_MACHINE>
    
#should we remove random parts of the header, i.e. checksum

#should I take out bias for RNNs?

#for the decoder,does the fork encoding need to happen ?
    #do we simply cat the hContext with the next words?
    
#Should the architecture just be encode, context and then prediction???

#Input data, should it have character and hex pair encoding as well?

In [1]:
%matplotlib inline

import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'#,optimizer=fast_compile'

import time
import os
import json
import subprocess
import cPickle
import sys
import binascii
import multiprocessing as mp
from itertools import chain
from collections import OrderedDict
import logging
#import ipaddress

import numpy as np
import random
from copy import copy

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP,\
                          Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, Fork, LSTM
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T


module_logger = logging.getLogger(__name__)
sys.setrecursionlimit(100000)


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


In [2]:
import datetime
import time
ts = time.time()
timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

# Data

In [3]:
#start = time.time()
dataPath = '/data/fs4/datasets/pcaps/smallFlows.pcap'
#pcaps = rdpcap(dataPath)
#sessionPrep = pcaps.sessions()
#end = time.time()
#print end - start

In [4]:
def parse_header(line):  # pragma: no cover
    ret_dict = {}
    h = line.split()
    if h[2] == 'IP6':
        """
        Conditional formatting based on ethernet type.
        IPv4 format: 0.0.0.0.port
        IPv6 format (one of many): 0:0:0:0:0:0.port
        """
        ret_dict['src_port'] = h[3].split('.')[-1]
        ret_dict['src_ip'] = h[3].split('.')[0]
        ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
        ret_dict['dest_ip'] = h[5].split('.')[0]
    else:
        if len(h[3].split('.')) > 4:
            ret_dict['src_port'] = h[3].split('.')[-1]
            ret_dict['src_ip'] = '.'.join(h[3].split('.')[:-1])
        else:
            ret_dict['src_ip'] = h[3]
            ret_dict['src_port'] = ''
        if len(h[5].split('.')) > 4:
            ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
            ret_dict['dest_ip'] = '.'.join(h[5].split('.')[:-1])
        else:
            ret_dict['dest_ip'] = h[5].split(':')[0]
            ret_dict['dest_port'] = ''
    return ret_dict


def parse_data(line):  # pragma: no cover
    ret_str = ''
    h, d = line.split(':', 1)
    ret_str = d.strip().replace(' ', '')
    return ret_str


def process_packet(output):  # pragma: no cover
    # TODO!! throws away the first packet!
    ret_header = {}
    ret_dict = {}
    ret_data = ''
    hasHeader = False
    for line in output:
        line = line.strip()
        if line:
            if not line.startswith('0x'):
                # header line
                if ret_dict and ret_data:
                    # about to start new header, finished with hex
                    ret_dict['data'] = ret_data
                    yield ret_dict
                    ret_dict.clear()
                    ret_header.clear()
                    ret_data = ''
                    hasHeader = False

                # parse next header
                try:
                    ret_header = parse_header(line)
                    ret_dict.update(ret_header)
                    hasHeader = True
                except:
                    ret_header.clear()
                    ret_dict.clear()
                    ret_data = ''
                    hasHeader = False

            else:
                # hex data line
                if hasHeader:
                    data = parse_data(line)
                    ret_data = ret_data + data
                else:
                    continue


def is_clean_packet(packet):  # pragma: no cover
    """
    Returns whether or not the parsed packet is valid
    or not. Checks that both the src and dest
    ports are integers. Checks that src and dest IPs
    are valid address formats. Checks that packet data
    is hex. Returns True if all tests pass, False otherwise.
    """
    if not packet['src_port'].isdigit(): return False
    if not packet['dest_port'].isdigit(): return False

    if packet['src_ip'].isalpha(): return False
    if packet['dest_ip'].isalpha(): return False

    if 'data' in packet:
        try:
            int(packet['data'], 16)
        except:
            return False

    return True


def order_keys(hexSessionDict):
    """
    Returns list of the hex sessions in (rough) time order.
    """
    orderedKeys = []

    for key in sorted(hexSessionDict.keys(), key=lambda key: hexSessionDict[key][1]):
        orderedKeys.append(key)

    return orderedKeys


def read_pcap(path):  # pragma: no cover
    print 'starting reading pcap file'
    hex_sessions = {} 
    proc = subprocess.Popen('tcpdump -nn -tttt -xx -r '+path,
                            shell=True,
                            stdout=subprocess.PIPE)
    insert_num = 0  # keeps track of insertion order into dict
    for packet in process_packet(proc.stdout):
        if not is_clean_packet(packet):
            continue
        if 'data' in packet:
            key = (packet['src_ip']+":"+packet['src_port'], packet['dest_ip']+":"+packet['dest_port'])
            rev_key = (key[1], key[0])
            if key in hex_sessions:
                hex_sessions[key][0].append(packet['data'])
            elif rev_key in hex_sessions:
                hex_sessions[rev_key][0].append(packet['data'])
            else:
                hex_sessions[key] = ([packet['data']], insert_num)
                insert_num += 1

    print 'finished reading pcap file'
    return hex_sessions

In [5]:
def pickleFile(thing2save, file2save2=None, filePath='/work/notebooks/drawModels/', fileName='myModels'):  # pragma: no cover

    if file2save2 is None:
        f = file(filePath+fileName+'.pickle', 'wb')
    else:
        f = file(filePath+file2save2, 'wb')

    cPickle.dump(thing2save, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()


def loadFile(filePath):  # pragma: no cover
    file2open = file(filePath, 'rb')
    loadedFile = cPickle.load(file2open)
    file2open.close()
    return loadedFile


def removeBadSessionizer(hexSessionDict, saveFile=False, dataPath=None, fileName=None):  # pragma: no cover
    for ses in hexSessionDict.keys():
        paclens = []
        for pac in hexSessionDict[ses][0]:
            paclens.append(len(pac))
        if np.min(paclens)<80:
            del hexSessionDict[ses]

    if saveFile:
        print 'pickling sessions'
        pickleFile(hexSessionDict, filePath=dataPath, fileName=fileName)

    return hexSessionDict


# Making the hex dictionary
def hexTokenizer():  # pragma: no cover
    hexstring = '''0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F,
                   10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1A, 1B,
                   1C, 1D, 1E, 1F, 20, 21, 22, 23, 24, 25, 26, 27,
                   28, 29, 2A, 2B, 2C, 2D, 2E, 2F, 30, 31, 32, 33,
                   34, 35, 36, 37, 38, 39, 3A, 3B, 3C, 3D, 3E, 3F,
                   40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 4A, 4B,
                   4C, 4D, 4E, 4F, 50, 51, 52, 53, 54, 55, 56, 57,
                   58, 59, 5A, 5B, 5C, 5D, 5E, 5F, 60, 61, 62, 63,
                   64, 65, 66, 67, 68, 69, 6A, 6B, 6C, 6D, 6E, 6F,
                   70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 7A, 7B,
                   7C, 7D, 7E, 7F, 80, 81, 82, 83, 84, 85, 86, 87,
                   88, 89, 8A, 8B, 8C, 8D, 8E, 8F, 90, 91, 92, 93,
                   94, 95, 96, 97, 98, 99, 9A, 9B, 9C, 9D, 9E, 9F,
                   A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, AA, AB,
                   AC, AD, AE, AF, B0, B1, B2, B3, B4, B5, B6, B7,
                   B8, B9, BA, BB, BC, BD, BE, BF, C0, C1, C2, C3,
                   C4, C5, C6, C7, C8, C9, CA, CB, CC, CD, CE, CF,
                   D0, D1, D2, D3, D4, D5, D6, D7, D8, D9, DA, DB,
                   DC, DD, DE, DF, E0, E1, E2, E3, E4, E5, E6, E7,
                   E8, E9, EA, EB, EC, ED, EE, EF, F0, F1, F2, F3,
                   F4, F5, F6, F7, F8, F9, FA, FB, FC, FD, FE, FF'''.replace('\t', '')

    hexList = [x.strip() for x in hexstring.lower().split(',')]
    hexList.append('<EOP>')  # End Of Packet token
    hexDict = {}

    for key, val in enumerate(hexList):
        if len(val) == 1:
            val = '0'+val
        hexDict[val] = key  #dictionary k=hex, v=int  

    return hexDict

In [6]:
hexSessions = read_pcap(dataPath)
hexSessions = removeBadSessionizer(hexSessions)
hexSessionsKeys = order_keys(hexSessions)
hexDict = hexTokenizer()

starting reading pcap file
finished reading pcap file


# Dictionary of IP communications

In [7]:
def oneHot(index, granular = 'hex'):
    if granular == 'hex':
        vecLen = 257
    else:
        vecLen = 17
    
    zeroVec = np.zeros(vecLen)
    zeroVec[index] = 1.0
    
    return zeroVec
 

In [8]:

def oneSessionEncoder(sessionPackets, hexDict, maxPackets=2, packetTimeSteps=100,
                      packetReverse=False, charLevel=False, padOldTimeSteps=True):  # pragma: no cover
    
    sessionCollect = []
    packetCollect = []
    
    if charLevel:
        vecLen = 17
    else:
        vecLen = 257
    
    if len(sessionPackets) > maxPackets: #crop the number of sessions to maxPackets
        sessionList = copy(sessionPackets[:maxPackets])
    else:
        sessionList = copy(sessionPackets)

    for packet in sessionList:
        packet = packet[32:36]+packet[44:46]+packet[46:48]+packet[52:60]+packet[60:68]+\
                 packet[68:70]+packet[70:72]+packet[72:74]
        packet = [hexDict[packet[i:i+2]] for i in xrange(0,len(packet)-2+1,2)]
            
        if len(packet) >= packetTimeSteps: #crop packet to length packetTimeSteps
            packet = packet[:packetTimeSteps]
            packet = packet+[256] #add <EOP> end of packet token
        else:
            packet = packet+[256] #add <EOP> end of packet token
        
        packetCollect.append(packet)
        
        pacMat = np.array([oneHot(x) for x in packet]) #one hot encoding of packet into a matrix
        pacMatLen = len(pacMat)
        
        #padding packet
        if packetReverse:
            pacMat = pacMat[::-1]

        if pacMatLen < packetTimeSteps:
            #pad by stacking zeros on top of data so that earlier timesteps do not have information
            #padding the packet such that zeros are after the actual info for better translation
            if padOldTimeSteps:
                pacMat = np.vstack( ( np.zeros((packetTimeSteps-pacMatLen,vecLen)), pacMat) ) 
            else:
                pacMat = np.vstack( (pacMat, np.zeros((packetTimeSteps-pacMatLen,vecLen))) ) 

        if pacMatLen > packetTimeSteps:
            pacMat = pacMat[:packetTimeSteps, :]

        sessionCollect.append(pacMat)

    #padding session
    sessionCollect = np.asarray(sessionCollect, dtype=theano.config.floatX)
    numPacketsInSession = sessionCollect.shape[0]
    if numPacketsInSession < maxPackets:
        #pad sessions to fit the
        sessionCollect = np.vstack( (sessionCollect,np.zeros((maxPackets-numPacketsInSession,
                                                             packetTimeSteps, vecLen))) )
    
    return sessionCollect, packetCollect

# Learning functions

In [9]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def dropout(X, p=0.):
    if p != 0:
        retain_prob = 1 - p
        X = X / retain_prob * srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
    return X

# Gradient clipping
def clip_norm(g, c, n): 
    '''n is the norm, c is the threashold, and g is the gradient'''
    
    if c > 0: 
        g = T.switch(T.ge(n, c), g*c/n, g) 
    return g

def clip_norms(gs, c):
    norm = T.sqrt(sum([T.sum(g**2) for g in gs]))
    return [clip_norm(g, c, norm) for g in gs]

# Regularizers
def max_norm(p, maxnorm = 0.):
    if maxnorm > 0:
        norms = T.sqrt(T.sum(T.sqr(p), axis=0))
        desired = T.clip(norms, 0, maxnorm)
        p = p * (desired/ (1e-7 + norms))
    return p

def gradient_regularize(p, g, l1 = 0., l2 = 0.):
    g += p * l2
    g += T.sgn(p) * l1
    return g

def weight_regularize(p, maxnorm = 0.):
    p = max_norm(p, maxnorm)
    return p

def Adam(params, cost, lr=0.0002, b1=0.1, b2=0.001, e=1e-8, l1 = 0., l2 = 0., maxnorm = 0., c = 8):
    
    updates = []
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    
    i = theano.shared(floatX(0.))
    i_t = i + 1.
    fix1 = 1. - b1**(i_t)
    fix2 = 1. - b2**(i_t)
    lr_t = lr * (T.sqrt(fix2) / fix1)
    
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        g_t = gradient_regularize(p, g_t, l1=l1, l2=l2)
        p_t = p - (lr_t * g_t)
        p_t = weight_regularize(p_t, maxnorm=maxnorm)
        
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    
    updates.append((i, i_t))
    #if iteration%100 == 0:
    #    updates.append((lr, lr*0.93))
    #else:
    #    updates.append((lr, lr))
    
    return updates

def RMSprop(cost, params, lr = 0.001, l1 = 0., l2 = 0., maxnorm = 0., rho=0.9, epsilon=1e-6, c = 8):
    
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    updates = []
    
    for p, g in zip(params, grads):
        g = gradient_regularize(p, g, l1 = l1, l2 = l2)
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        updates.append((acc, acc_new))
        
        updated_p = p - lr * (g / T.sqrt(acc_new + epsilon))
        updated_p = weight_regularize(updated_p, maxnorm = maxnorm)
        updates.append((p, updated_p))
    return updates

In [9]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def dropout(X, p=0.):
    if p != 0:
        retain_prob = 1 - p
        X = X / retain_prob * srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
    return X

# Gradient clipping
def clip_norm(g, c, n): 
    '''n is the norm, c is the threashold, and g is the gradient'''
    
    if c > 0: 
        g = T.switch(T.ge(n, c), g*c/n, g) 
    return g

def clip_norms(gs, c):
    norm = T.sqrt(sum([T.sum(g**2) for g in gs]))
    return [clip_norm(g, c, norm) for g in gs]

# Regularizers
def max_norm(p, maxnorm = 0.):
    if maxnorm > 0:
        norms = T.sqrt(T.sum(T.sqr(p), axis=0))
        desired = T.clip(norms, 0, maxnorm)
        p = p * (desired/ (1e-7 + norms))
    return p

def gradient_regularize(p, g, l1 = 0., l2 = 0.):
    g += p * l2
    g += T.sgn(p) * l1
    return g

def weight_regularize(p, maxnorm = 0.):
    p = max_norm(p, maxnorm)
    return p

def Adam(params, cost, lr=0.0002, b1=0.1, b2=0.001, e=1e-8, l1 = 0., l2 = 0., maxnorm = 0., c = 8):
    
    updates = []
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    
    i = theano.shared(floatX(0.))
    i_t = i + 1.
    fix1 = 1. - b1**(i_t)
    fix2 = 1. - b2**(i_t)
    lr_t = lr * (T.sqrt(fix2) / fix1)
    
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        g_t = gradient_regularize(p, g_t, l1=l1, l2=l2)
        p_t = p - (lr_t * g_t)
        p_t = weight_regularize(p_t, maxnorm=maxnorm)
        
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    
    updates.append((i, i_t))
    #if iteration%100 == 0:
    #    updates.append((lr, lr*0.93))
    #else:
    #    updates.append((lr, lr))
    
    return updates

def RMSprop(cost, params, lr = 0.001, l1 = 0., l2 = 0., maxnorm = 0., rho=0.9, epsilon=1e-6, c = 8):
    
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    updates = []
    
    for p, g in zip(params, grads):
        g = gradient_regularize(p, g, l1 = l1, l2 = l2)
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        updates.append((acc, acc_new))
        
        updated_p = p - lr * (g / T.sqrt(acc_new + epsilon))
        updated_p = weight_regularize(updated_p, maxnorm = maxnorm)
        updates.append((p, updated_p))
    return updates

# Unsupervised feature extractor

### Initialization for both the unsupervised net and the classifier

In [24]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [10]:
X = T.tensor4('inputs')
Y = T.matrix('targets')

wtstd = 0.2
dimIn = 257 #hex has 256 characters + the <EOP> character
dim = 100 #dimension reduction size
rnnType = 'gru' #gru or lstm
bidirectional = False
linewt_init = IsotropicGaussian(wtstd)
line_bias = Constant(1.0)
rnnwt_init = IsotropicGaussian(wtstd)
rnnbias_init = Constant(0.0)
packetReverse = False

###ENCODER
if rnnType == 'gru':
    rnn = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIn, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)


###CONTEXT
if rnnType == 'gru':
    rnnContext = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                biases_init = rnnbias_init, name = 'gruContext')
else:
    rnnContext = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                      name = 'lstmContext')

forkContext = Fork(output_names=['linearContext', 'gatesContext'],
            name='forkContext', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

if bidirectional:
    dimDec = dim*2
    
    if rnnType == 'gru':
        rnnContextRev = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                       biases_init = rnnbias_init, name = 'gruContextRev')
        
    else:
        rnnContextRev = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init,
                             name = 'lstmContextRev')
    
    rnnContextRev.initialize()

else:
    dimDec = dim


###DECODER
if rnnType == 'gru':
    rnnDec = GatedRecurrent(dim=dimIn, weights_init = rnnwt_init, 
                            biases_init = rnnbias_init, name = 'gruDecoder')
else:
    rnnDec = LSTM(dim=dimIn, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstmDecoder')


forkDec = Fork(output_names=['linear', 'gates'],
            name='forkDec', input_dim=dimDec, output_dims=[dim, dim*dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

forkFinal = Fork(output_names=['linear', 'gates'],
            name='forkFinal', input_dim=dim, output_dims=[dimIn, dimIn*dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

#initialize the weights in all the functions
fork.initialize()
rnn.initialize()

forkContext.initialize()
rnnContext.initialize()
forkDec.initialize()

forkFinal.initialize()
rnnDec.initialize()

### Unsupervised graph

In [11]:
#This section can be skipped if you want to go directly to the classifier

learning_rate = theano.shared(np.array(0.001, dtype=theano.config.floatX))
learning_decay = np.array(0.9, dtype=theano.config.floatX)
batch_size = 20
maxPackets = 3
packetTimeSteps = 16
attention = False

def onestepEnc(X):
        data1, data2 = fork.apply(X) 

        if rnnType == 'gru':
            hEnc = rnn.apply(data1, data2) 
        else:
            hEnc, _ = rnn.apply(data2)

        return hEnc, data1

[hEnc, data1], _ = theano.scan(onestepEnc, X) #(mini*numPackets, packetLen, 1, hexdictLen)
hEncReshape = T.reshape(hEnc[:,-1], (-1, maxPackets, 1, dim)) #[:,-1] takes the last rep for each packet
                                                             #(mini, numPackets, 1, dimReduced)
def onestepContext(hEncReshape):

    data3, data4 = forkContext.apply(hEncReshape)

    if rnnType == 'gru':
        hContext = rnnContext.apply(data3, data4)
    else:
        hContext, _ = rnnContext.apply(data4)

    return hContext

hContext, _ = theano.scan(onestepContext, hEncReshape)

if attention:
    print 'what are you doing?!?!'
else:
    data5, _ = forkDec.apply(hContext)
    


In [12]:
ctxtest = theano.function([X], [data1, hContext], allow_input_downcast=True)

In [13]:
testones = np.ones((12,16,1,257))
testones[0] = testones[0]+3
testones[-1] = testones[-1]-3

In [14]:
ctxtest(testones)[0].shape


(12, 16, 1, 100)

In [237]:
#(4, 2, 1, 100)
ctxtest(testones)[1][:,:-1]

(4, 2, 1, 100)

In [18]:
#.reshape((4, maxPackets, packetTimeSteps, dim)) = (4, 3, 16, 100)
#[:,1:,:-1] = (4, 2, 15, 100)
ctxtest(testones)[0].reshape((4, maxPackets, packetTimeSteps, dim))[:,1:,:-1]

array([[[[-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         ..., 
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197]],

        [[-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         [-3.22383356,  2.83601284, -0.62676251, ..., -2.74221468,
           1.89804673, -1.32940197],
         ..., 
         [-3.22383356,  2.836012

In [ ]:
#data1, (batch_size, maxPackets, packetTimeSteps, dim))[:,1:,:-1]),
#(12, 16, 1, 100)
ctxtest(testones)[0]

In [239]:
######START HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Transform hContext then add it to the concatenated zeros stuff

#(4, 2, 16, 100)
#                         (batch_size, maxPackets-1, 1, dim)
(np.concatenate((np.zeros((4,2,1,100)),ctxtest(testones)[0].reshape((4, maxPackets, packetTimeSteps, dim))[:,1:,:-1]), axis = 2) + ctxtest(testones)[1][:,:-1])

(4, 2, 16, 100)

In [153]:
from blocks.bricks.recurrent import BaseRecurrent, recurrent
from blocks.bricks.recurrent import SimpleRecurrent
from blocks import initialization
from blocks.bricks import Identity
#from blocks.initialization import Identity

#@recurrent
rnntest = SimpleRecurrent(
          dim=6, activation=Identity(), name='second_recurrent_layer',
          weights_init=initialization.Identity())

rnntest.initialize()

In [232]:
testdat1 = T.tensor3()
testdat2 = T.matrix()
testout = rnntest.apply(inputs = testdat1, states = testdat2, iterate = False)
#testout = rnntest.apply(inputs = testdat1, states = testdat2, iterate = False)

In [233]:
#testy = theano.function([testdat1], testout, allow_input_downcast=True)
testy = theano.function([testdat1, testdat2], testout, allow_input_downcast=True)

In [229]:
#testx = np.random.rand(4,1,6)
testx

array([[[ 0.55550689,  0.28341385,  0.15724478,  0.98274833,  0.78550856,
          0.54037572]],

       [[ 0.0051092 ,  0.63481661,  0.94328937,  0.10269005,  0.75664565,
          0.17752227]],

       [[ 0.50084736,  0.64411131,  0.01379403,  0.99626781,  0.1945023 ,
          0.03193511]],

       [[ 0.24559894,  0.11226367,  0.5846486 ,  0.10384701,  0.12228761,
          0.47314758]]])

In [234]:
testy(testx, np.ones((1,6), dtype=theano.config.floatX))

array([[[ 1.55550694,  1.28341389,  1.1572448 ,  1.98274827,  1.78550863,
          1.54037571]],

       [[ 1.00510919,  1.63481665,  1.9432894 ,  1.1026901 ,  1.75664568,
          1.1775223 ]],

       [[ 1.50084734,  1.64411139,  1.01379406,  1.9962678 ,  1.19450235,
          1.0319351 ]],

       [[ 1.24559891,  1.11226368,  1.58464861,  1.10384703,  1.12228763,
          1.47314763]]], dtype=float32)

In [174]:
testy(np.ones((3,1,6)), np.ones((3,1,6))+5)

array([[[[  8.,   8.,   8.,   8.,   8.,   8.]],

        [[  8.,   8.,   8.,   8.,   8.,   8.]],

        [[  8.,   8.,   8.,   8.,   8.,   8.]]],


       [[[ 14.,  14.,  14.,  14.,  14.,  14.]],

        [[ 14.,  14.,  14.,  14.,  14.,  14.]],

        [[ 14.,  14.,  14.,  14.,  14.,  14.]]]], dtype=float32)

In [211]:
testy(np.random.rand(4,1,6))

array([[[ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ],
        [ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ],
        [ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ],
        [ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ],
        [ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ],
        [ 0.42494223,  0.88823497,  0.27871415,  0.67081416,  0.47666129,
          0.6851989 ]],

       [[ 0.60906661,  1.23047972,  0.96374762,  1.46646965,  1.26869583,
          1.2148068 ],
        [ 0.60906661,  1.23047972,  0.96374762,  1.46646965,  1.26869583,
          1.2148068 ],
        [ 0.60906661,  1.23047972,  0.96374762,  1.46646965,  1.26869583,
          1.2148068 ],
        [ 0.60906661,  1.23047972,  0.96374762,  1.46646965,  1.26869583,
          1.2148068 ],
        [ 0.60906661,  1.230

In [ ]:
 #this fork makes data5 same dim as data1 (the orig word embedding)

#decoding data needs to be one timestep (next packet in session) ahead, thus data1 we ignore the first packet
#and the last hidden state of the context RNN.
#THINK about L2 pooling before cat
#THINK should we concatenate with X instead of data5
#if packetReverse:
#    data1 = data1[:,::-1]
#do we need data5??

data7 = T.concatenate((data5[:,:-1], 
                       T.reshape(data1, (batch_size, maxPackets, packetTimeSteps, dim))[:,1:,:-1]), 
                      axis = 2)
                      
                      #data1 is the original embedding of X, data5 is transformed context output
                      #get rid of first packet in data 5
                      #get rid of last context vector

#predicts all but the first packet, i.e. session[1:]
#output.shape = (20, 2, 100, 257), (minibatch, 
def onestepDec(data7):
    
    data8, data9 = forkFinal.apply(data7) #forkFinal transforms back to original dimIn
    
    if rnnType == 'gru':
        hDec = rnnDec.apply(data8, data9) 
    else:
        hDec, _ = rnnDec.apply(data9)
        #hDec = hinit #hDec shape = (batch_size*(maxPackets-1), packetTimeSteps, 257)
    
    return hDec


hDec, _ = theano.scan(onestepDec, data7)

hDecReshape = T.reshape(hDec, (batch_size*(maxPackets-1), packetTimeSteps, dimIn))


softmax = NDimensionalSoftmax()
softout = softmax.apply(hDecReshape, extra_ndim = 1)
predX = T.reshape(T.reshape(X,(batch_size, maxPackets, packetTimeSteps, dimIn))[:,1:,:,:], 
                  (batch_size*(maxPackets-1), packetTimeSteps, dimIn))

precost = predX*T.log(softout) + (1-predX)*T.log(1-softout)
precost2 = -T.sum(T.sum(precost, axis = 2), axis = 1)
#precost2 = -T.mean(T.sum(T.sum(precost, axis = 2), axis = 1))

cost = T.mean(precost2)
#cost = T.mean(BinaryCrossEntropy().apply(predX, softout))
cg = ComputationGraph([cost])

learning_rate = theano.shared(np.array(0.0001, dtype=theano.config.floatX))
learning_decay = np.array(0.9, dtype=theano.config.floatX)

params = VariableFilter(roles = [PARAMETER])(cg.variables)
updates = Adam(params, cost, learning_rate, c=1) #c is gradient clipping parameter
#updates = RMSprop(cost, params, learning_rate, c=1)

#gradients = T.grad(cost, params)
#gradients = clip_norms(gradients, 1)
#gradientFun = theano.function([X], gradients, allow_input_downcast=True)

print "compiling you beautiful person"
train = theano.function([X], [cost, hContext], updates = updates, allow_input_downcast=True)
predict = theano.function([X], [softout, hContext], allow_input_downcast=True)
print "finished compiling"

### Unsupervised training

In [ ]:
#randomize data
hexSessionsKeys = hexSessions.keys()
#random.shuffle(hexSessionsKeys)
trainPercent = 0.9
trainIndex = int(len(hexSessionsKeys)*trainPercent)

padOldTimeSteps = False

runname = 'hred'
epochCost = []
gradNorms = []
contextCollect = []

epochs = 80
iteration = 0



for epoch in xrange(epochs):
    costCollect = []
    
    for start, end in zip(range(0, trainIndex,batch_size), range(batch_size, trainIndex, batch_size)):
        
        trainingSessions = []
        
        for trainKey in range(start, end):
            sessionForEncoding = list(hexSessions[hexSessions.keys()[trainKey]])
            
            #encode a normal session
            #oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
            #                              padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
            #                              packetTimeSteps = packetTimeSteps)
            #trainingSessions.append(oneHotSes[0])
            #trainingTargets.append(normalTarget)
            
            #encode an abby normal session
            
            oneHotSes = oneSessionEncoder(sessionForEncoding,
                                              hexDict = hexDict,
                                              packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, 
                                              maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)
            
            trainingSessions.append(oneHotSes[0])
            
        sessionsMinibatch = np.asarray(trainingSessions).reshape((batch_size*maxPackets, packetTimeSteps, 1, dimIn))        
    
        costfun = train(sessionsMinibatch)
        costCollect.append(costfun[0])
        
        
        
        
        if iteration == 0:
            print 'you are amazing'
            
        iteration+=1
        
        #if iteration%80 == 0:
        #    learning_rate.set_value(learning_rate.get_value() * learning_decay)
        #    print '   learning rate: ', learning_rate.get_value()
        
    ####SAVE COST TO FILE 
    
    if epoch%2 == 0:
                
        print(' ')
        print 'Epoch: ', epoch
        epochCost.append(np.mean(costCollect))
        contextCollect.append(costfun[1][:4])
        print 'Epoch cost average: ', epochCost[-1]
        #grads = gradientFun(inputs, outputs)
        #for gra in grads:
        #    print '  gradient norms: ', np.linalg.norm(gra)
        
    #np.savetxt(runname+"_COST.csv", epochCost, delimiter=",")

In [ ]:
#Two huge extension:
#    1) give encoder to all decoding time steps
#    2) give predicted output to next time step as well
#    CHECK 3) use bidirectional to use give beginning and end of encoder to decoder


#294.692

In [ ]:
np.random.rand(3,4)+np.array([1,2,3,4])

In [ ]:
#Epoch:  32
#Epoch cost average:  311.735

In [ ]:
hexSessions[hexSessions.keys()[23]]

In [ ]:
tester = oneSessionEncoder(hexSessions[hexSessions.keys()[23]],hexDict = hexDict,
                                              packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, 
                                              maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)[0]
testercheck = oneSessionEncoder(hexSessions[hexSessions.keys()[23]],hexDict = hexDict,
                                              packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, 
                                              maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)[1]

In [ ]:
testerplus = []
for i in range(20):
    testerplus.append(tester)

In [ ]:
np.asarray(testerplus).shape

In [ ]:
sessionsMinibatch.shape

In [ ]:
testout = np.squeeze(costfun[1])[:,-1]

In [ ]:
plt.plot(testout[0], 'b', testout[19], 'r')
plt.show()

In [ ]:
(testout[0]-testout[1])*89989

In [ ]:
testout[1]

In [ ]:
np.argmax(predict(np.asarray(testerplus).reshape((batch_size*maxPackets, packetTimeSteps, 1, dimIn)))[:2], axis = 2)

In [ ]:
testercheck